# Model Inputs

### Packages

In [1]:
# General
import pandas as pd
import numpy as np
import pandas as pd
import os
DATA_DIR = "~/Desktop/code/data/"

# Pipeline
from sklearn.pipeline import Pipeline

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler 

from sklearn.decomposition import PCA 
import umap
import hdbscan

from sklearn.metrics import silhouette_score
from sklearn.cluster import KMeans

# Visualization
import plotly.express as px
import plotly.graph_objects as go

### Dataset

In [2]:
df = pd.read_csv(os.path.join(DATA_DIR, "ml_input.csv"), low_memory=False)

filter = df["ec"].str.contains("-") # filter non-specific ec numbers, e.g., 1.1.1.-
df = df[~filter]

df = df[["taxon_id", "media_id", "ec"]].value_counts().reset_index()
df = df.pivot(index=["taxon_id", "media_id"], columns="ec", values="count")
df = df.fillna(0.0).reset_index()

df.head()

ec,taxon_id,media_id,1.1.1.1,1.1.1.100,1.1.1.102,1.1.1.103,1.1.1.107,1.1.1.108,1.1.1.11,1.1.1.110,...,7.6.2.12,7.6.2.13,7.6.2.14,7.6.2.15,7.6.2.16,7.6.2.2,7.6.2.5,7.6.2.7,7.6.2.8,7.6.2.9
0,1002526,J22,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1004166,1a,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1004261,J181,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1004261,J455,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1005925,J118,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [3]:
# Filtering into training and test sets (database set vs. MAGs)
bins = df["media_id"].str.contains("unknown")
df_train = df[~bins]
df_test = df[bins]

# Model

### Training and test sets

In [4]:
# Split the datasets into training and test sets
TARGET = "media_id" # target label
RDM = 47 # seed for random_state

# Test case with MAGs
X_train = df_train.drop(["taxon_id", "media_id"], axis=1)
y_train = df_train[TARGET]
X_test = df_test.drop(["taxon_id", "media_id"], axis=1)
y_test = df_test[TARGET] 

### OR ###

# Model validation with only labeled samples from data.ipynb
#X_train, X_test, y_train, y_test = train_test_split(
#    df_train.drop(["taxon_id", "media_id"], axis=1),
#    df_train[TARGET],
#    test_size=0.2,
#    random_state=RDM
#)

### Pipeline

In [37]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import make_scorer, silhouette_score
import umap
import numpy as np

# Seed for reproducibility
RDM = 47

# Define the pipeline
pipeline = Pipeline([
    ('scaler', StandardScaler()),  # Optional: scale data
    ('pca', PCA(n_components=120)),  # Reduce dimensionality with PCA
    ('umap', umap.UMAP(  # UMAP step
        metric="euclidean",
        n_epochs=200,
        random_state=RDM,
        n_jobs=1
    ))
])

# Define parameter grid with correct step names
param_distributions = {
    'umap__n_components': [2, 5, 10, 15, 20, 25, 30, 50],
    'umap__n_neighbors': [10, 15, 25, 30, 35, 40, 45, 50], 
    'umap__min_dist': [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.7, 0.9]
}

# Custom scorer function
def umap_silhouette(X, y):
    embedding = X  # X here is already the transformed data by UMAP
    return silhouette_score(embedding, y)

# Wrap the custom scorer
scorer = make_scorer(umap_silhouette, greater_is_better=True)

# Initialize RandomizedSearchCV
search = RandomizedSearchCV(
    estimator=pipeline,
    param_distributions=param_distributions,
    n_iter=50,
    cv=5,
    random_state=RDM,
    n_jobs=-1,
    scoring=scorer
)

# Fit the search to your data
search.fit(X_train, y_train)

# Retrieve the best parameters
best_params = search.best_params_
best_model = search.best_estimator_

print("Best Parameters:", best_params)


c:\Users\jakel\anaconda3\envs\media\Lib\site-packages\sklearn\model_selection\_search.py:1052: UserWarning:

One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan]



Best Parameters: {'umap__n_neighbors': 50, 'umap__n_components': 20, 'umap__min_dist': 0.9}


In [51]:
# sklearn pipeline helps prevent data leakage; incorporate individual steps here
pipeline = Pipeline([
    ('scaler', StandardScaler()),    # Scale data (optional)
    ('pca', PCA(n_components=120)),   # Reduce dimensionality with PCA
    ('umap', umap.UMAP(              # Further reduce with UMAP
        metric="euclidean",
        n_components=2, #20 for clustering, 2 for visualization
        n_epochs=200, #200 recommended for large datasets, higher => stricter clustering
        random_state=RDM,
        n_jobs=1,
        n_neighbors=50,
        min_dist=0.2
        )) 
])

### Fit and transform

In [52]:
# Fit and transform the pipeline on the training and test data
X_train_transformed = pipeline.fit_transform(X_train) # fit pipeline and transform training data
X_test_transformed = pipeline.transform(X_test) # transform test data

### Optimizing n_clusters

In [53]:
# Silhouette coefficient method
from sklearn.metrics import silhouette_score
from sklearn.cluster import KMeans

# Test a range of clusters for their silhouette coefficients
clusters = []
for n_cluster in range(20, 100):
    kmeans = KMeans(n_clusters=n_cluster).fit(X_train_transformed)
    label = kmeans.labels_
    sil_coeff = silhouette_score(X_train_transformed, label, metric='euclidean')
    clusters.append({'n_clusters': n_cluster, 'coefficient': sil_coeff})

# Select the maximum coefficient
clusters = pd.DataFrame(clusters)
c = clusters.iloc[clusters["coefficient"].argmax()]["n_clusters"]
s = clusters["coefficient"].max()
n = int(c)

print("{} clusters returns a maximum Silhouette Coefficient of {}".format(n-1, s))

97 clusters returns a maximum Silhouette Coefficient of 0.6940653920173645


In [54]:
# Cluster
clusterer = KMeans(n_clusters=n, random_state=RDM)
cluster_labels = clusterer.fit_predict(X_train_transformed)
test_clusters = clusterer.predict(X_test_transformed)

### Metrics

In [55]:
# Initial dimensionality reduction performance (PCA)
pca = pipeline.named_steps['pca']
explained_variance = pca.explained_variance_ratio_
print(f"Total PCA explained variance: {explained_variance.sum()}")

# Further dimensionality reduction and clustering performance
silhouette_avg = silhouette_score(X_train_transformed, cluster_labels)
print(clusterer.labels_.max(), "KMeans training clusters")
print(f"Average Silhouette Score on KMeans clusters: {silhouette_avg}")
    # Silhouette score <=0.25 implies poor clustering, 0.25<x<=0.50 is fair, <50 is good

Total PCA explained variance: 0.8273301831732849
97 KMeans training clusters
Average Silhouette Score on KMeans clusters: 0.680009663105011


In [56]:
# Compare to hdbscan clustering 

# Apply HDBSCAN clustering on the transformed training data
hclusterer = hdbscan.HDBSCAN(
    min_samples=5, 
    min_cluster_size=15, 
    #gen_min_span_tree=True, 
    prediction_data=True # cluster_selection_method="leaf", for more fine-grained clustering
)
hcluster_labels = hclusterer.fit_predict(X_train_transformed)

# Clustering performance (UMAP and hdbscan)
hsilhouette_avg = silhouette_score(X_train_transformed, hcluster_labels)
print(hclusterer.labels_.max(), "hdbscan training clusters")
print(f"Silhouette Score on hdbscan clusters: {hsilhouette_avg}")

# Convert to numpy array for hdbscan and predict clustering
#test_points = test_df[["Component 1", "Component 2"]].to_numpy()
#test_labels, strengths = hdbscan.approximate_predict(clusterer, test_points)

# Optional constraints
#filtered = train_df[train_df['Cluster'] >= 0]
#test_labels = clusterer.fit_predict(X_test_transformed)

163 hdbscan training clusters
Silhouette Score on hdbscan clusters: 0.5721282958984375


### Visualization

In [57]:
# Create DataFrames for plotting
train_df = pd.DataFrame(X_train_transformed, columns=['Component 1', 'Component 2'])
train_df['Set'] = 'Training'
train_df['Target'] = y_train
train_df['Cluster'] = cluster_labels # only showing training clusters for now, mapping test over these
train_df['taxon_id'] = df_train["taxon_id"]
train_df.head()

,Component 1,Component 2,Set,Target,Cluster,taxon_id
0,-7.759715,20.165302,Training,11,18,1002526
1,-11.117567,12.047003,Training,14,7,1004166
2,26.122124,-11.286343,Training,10,13,1004261
3,26.130684,-11.296138,Training,10,13,1004261
4,-7.752337,20.130255,Training,11,18,1005925


In [60]:
test_df = pd.DataFrame(X_test_transformed, columns=['Component 1', 'Component 2'])
test_df['Set'] = 'Test'
test_df['Target'] = list(df_test['media_id'])
test_df['Cluster'] = test_clusters
test_df['taxon_id'] = list(df_test['taxon_id'])
test_df.head()

,Component 1,Component 2,Set,Target,Cluster,taxon_id
0,6.791389,-1.248185,Test,unknown,0,KBase_derived_Bin.001.fastaBA_F_extracted_bins...
1,6.925846,-1.909681,Test,unknown,79,KBase_derived_Bin.002.fastaBA_F_extracted_bins...
2,7.118182,-0.700613,Test,unknown,0,KBase_derived_Bin.004.fastaBA_F_extracted_bins...
3,7.616117,-0.845554,Test,unknown,70,KBase_derived_Bin.005.fastaBA_F_extracted_bins...
4,6.877962,-0.991795,Test,unknown,0,KBase_derived_Bin.006.fastaBA_F_extracted_bins...


In [61]:
# Plot the training points
fig = px.scatter(
    data_frame = train_df,
    x = "Component 1",
    y = "Component 2",
    color = "Cluster",
    hover_data = ["taxon_id", "Target"],
    opacity = 0.3
) 

# Add the test points as a scatter trace
fig.add_trace(
    go.Scatter(
        x=test_df["Component 1"],  # x coordinates
        y=test_df["Component 2"],  # y coordinates
        mode='markers',
        marker=dict(
            color='black',  
            size=5,  
            opacity=1.0 
        ),
        text=test_df["Target"]  # marker hover text
    )
)

# Opacity slider
steps = []
for i in range(11):  # 0 to 1 in steps of 0.1
    step = dict(
        method="restyle",
        args=[{"marker.opacity": [i/20]}, [0]],  # Adjust opacity of the first trace only
        label=str(i/20)
    )
    steps.append(step)
sliders = [dict(
    active=3,  # initial value corresponding to opacity=0.3
    currentvalue={"prefix": "Opacity: "},
    pad={"t": 50},
    steps=steps
)]

fig.update_layout(
    sliders=sliders,
    title="KMeans",
    template="plotly_white"
)

fig.show()

# Classifier

### Nearest Neighbors

In [63]:
#from sklearn.linear_model import LogisticRegression
#from scipy.stats import uniform

#from sklearn.model_selection import RandomizedSearchCV
#RandomizedSearchCV(KNearestNeighbors(),
#                                 param_grid={'knearestneighbors__n_neighbors': [5, 10, 40]},
#                                 cv=5,
#                                 refit=True)

In [64]:
from sklearn.neighbors import KNeighborsClassifier

y_train = cluster_labels
y_test = test_clusters 

neigh = KNeighborsClassifier(n_neighbors=3)
neigh.fit(X_train_transformed, y_train)

print(neigh.predict(X_test_transformed))
print(neigh.predict_proba(X_test_transformed))

[ 0 79  0 70  0  0  0  0 79  0  0 79 77  0 70 79 70 79 22  0  0 70 79]
[[1. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 ...
 [1. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [72]:
#TODO: remap with X_train/test and the CLUSTER LABELS, not this

from sklearn.neighbors import NearestNeighbors

# Find nearest neighbors and their distances
n_neighbors = 5 
nn_model = NearestNeighbors(n_neighbors=n_neighbors, algorithm='auto')
nn_model.fit(X_train_transformed, y_train)
distances, indices = nn_model.kneighbors(X_test_transformed)

In [73]:
# Add nearest neighbor indices to test dataframe
output_df = test_df.copy()
nn_df = pd.DataFrame(indices)
output_df['indices'] = nn_df[nn_df.columns[0:]].apply(lambda x: ';'.join(x.dropna().astype(str)), axis=1)
output_df["KNN_label"] = neigh.predict(X_test_transformed)

# Explode the indices
output_df["indices"] = output_df["indices"].str.split(';')
output_df = output_df.explode("indices")

# Add indices as column for the training dataframe
train_index = train_df.copy()
train_index["indices"] = train_index.index
train_index["indices"] = train_index["indices"].astype(str)

# Merge our dataframes to provide labels
left = output_df.copy()
right = train_index[["indices", "Target", "Cluster", "taxon_id"]]
right = right.rename(columns={"Target": "neighbor_media_id", "taxon_id": "neighbor_taxon_id", "Cluster": "neighbor_cluster"})
merged = pd.merge(left, right, on="indices", how="left")



#merged.to_csv(os.path.join(DATA_DIR, "ml-model-output.csv"), index=False)
merged

,Component 1,Component 2,Set,Target,Cluster,taxon_id,indices,KNN_label,neighbor_media_id,neighbor_cluster,neighbor_taxon_id
0,6.791389,-1.248185,Test,unknown,0,KBase_derived_Bin.001.fastaBA_F_extracted_bins...,1062,0,0,0,1528
1,6.791389,-1.248185,Test,unknown,0,KBase_derived_Bin.001.fastaBA_F_extracted_bins...,1063,0,0,0,1528
2,6.791389,-1.248185,Test,unknown,0,KBase_derived_Bin.001.fastaBA_F_extracted_bins...,7485,0,64,0,"[{'NCBI tax id': 258, 'Matching level': 'speci..."
3,6.791389,-1.248185,Test,unknown,0,KBase_derived_Bin.001.fastaBA_F_extracted_bins...,6128,0,68,0,837
4,6.791389,-1.248185,Test,unknown,0,KBase_derived_Bin.001.fastaBA_F_extracted_bins...,4093,0,68,0,37923
...,...,...,...,...,...,...,...,...,...,...,...
110,6.753847,-1.782246,Test,unknown,79,KBase_derived_Bin.028.fastaBA_F_extracted_bins...,2719,79,68,79,2164
111,6.753847,-1.782246,Test,unknown,79,KBase_derived_Bin.028.fastaBA_F_extracted_bins...,832,79,68,79,145262
112,6.753847,-1.782246,Test,unknown,79,KBase_derived_Bin.028.fastaBA_F_extracted_bins...,2103,79,68,79,1884223
113,6.753847,-1.782246,Test,unknown,79,KBase_derived_Bin.028.fastaBA_F_extracted_bins...,7438,79,68,79,"[{'NCBI tax id': 2277, 'Matching level': 'spec..."


### Random Forest

In [74]:
y_train = cluster_labels
y_test = test_clusters

In [75]:
import sklearn
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

model = RandomForestClassifier(
    n_estimators=100,
    n_jobs=-1,
    random_state=RDM
)

model.fit(X_train_transformed, y_train)

RandomForestClassifier(n_jobs=-1, random_state=47)

In [76]:
report = classification_report(
    y_true=y_test,
    y_pred=model.predict(X_test_transformed),
    zero_division="warn"
)

print(report)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        10
          22       1.00      1.00      1.00         1
          70       1.00      1.00      1.00         5
          77       1.00      1.00      1.00         1
          79       1.00      1.00      1.00         6

    accuracy                           1.00        23
   macro avg       1.00      1.00      1.00        23
weighted avg       1.00      1.00      1.00        23



In [77]:
from sklearn import metrics

roc_score = metrics.roc_auc_score(
    y_true=y_test,
    y_score=model.predict_proba(X_test_transformed),
    labels=np.unique(y_train),
    average="weighted",
    multi_class="ovo"
)

print("ROC AUC score:", roc_score)

ROC AUC score: 1.0


In [78]:
#TODO: Cluster media by type and use as color labels in visualization
#TODO: Fix AUC-ROC score shape mismatch between training and test sets (maybe report train_test_split value, as extremely small sample size leads to 1.0)